In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd "/content/drive/My Drive/major"

/content/drive/My Drive/major


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [0]:
import psutil
import humanize
import os
import GPUtil as GPU

# XXX: only one GPU on Colab and isn’t guaranteed


In [0]:
GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()



Gen RAM Free: 12.3 GB  | Proc size: 899.4 MB
GPU RAM Free: 14858MB | Used: 221MB | Util   1% | Total 15079MB


In [0]:
!pip install soundfile
!pip install librosa
!pip install imageio

In [0]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
import scipy.signal as sig
import matplotlib.pyplot as plt
import librosa
import librosa.display
import scipy
import imageio

In [0]:
raw_data=np.load('all_sig_waves.npy',allow_pickle=True)
data=[]
labels=[]
for i in raw_data:
  data.append(i[0])
  labels.append(i[1])
del raw_data
outputdir='MFCC'

samplerate=22050
slice_len=3
only_mfcc=3
target= (32,128) # chek

In [0]:
import gc
gc.collect()

0

In [0]:
mfcc_data=[]
c=1
for i in data:
    print(c,len(data))
    if only_mfcc==1:
      mfccc = librosa.feature.mfcc(i, samplerate,n_mfcc=target[0])
      mfccc= scipy.misc.imresize(mfccc, target)
      mfccc = mfccc.reshape(32,128,1)
      mfcc_data.append(mfccc)
    elif only_mfcc==2:
      """
      Remove before pushing:
      The delta width is in 2K + 1 form
      Han et. al [16] propose widths ∆3, ∆11 and ∆19 (i.e., K = 1, K = 5, K = 9). The
      input to the CNN is then the vector, or four channel image, (static, ∆3, ∆11, ∆19),
      where static refers to the MFCCs.
      """
      mfccc = librosa.feature.mfcc(i,samplerate, n_mfcc=target[0])
      delta3_mfcc = librosa.feature.delta(mfccc, width=3, order=1)
      delta11_mfcc = librosa.feature.delta(mfccc, width=11, order=1)
      delta19_mfcc = librosa.feature.delta(mfccc, width=19, order=1)


      mfccc = scipy.misc.imresize(mfccc, target)
      delta3_mfcc = scipy.misc.imresize(delta3_mfcc, target)
      delta11_mfcc = scipy.misc.imresize(delta11_mfcc, target)
      delta19_mfcc = scipy.misc.imresize(delta19_mfcc, target)

      mfccc = mfccc.reshape(mfccc.shape[0], mfccc.shape[1], 1) #32,128,1
      delta3_mfcc = delta3_mfcc.reshape(delta3_mfcc.shape[0], delta3_mfcc.shape[1], 1)
      delta11_mfcc = delta11_mfcc.reshape(delta11_mfcc.shape[0], delta11_mfcc.shape[1], 1)
      delta19_mfcc = delta19_mfcc.reshape(delta19_mfcc.shape[0], delta19_mfcc.shape[1], 1)

      mfccc_delta = np.concatenate([mfccc, delta3_mfcc, delta11_mfcc, delta19_mfcc], axis=2)
      mfcc_data.append(mfccc_delta)
    else:
      stft = np.abs(librosa.stft(i))
      #print(stft.shape)
      mfccs = np.mean(librosa.feature.mfcc(y=i, sr=samplerate, n_mfcc=32).T,axis=0)
      #print(mfccs.shape)
      chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=samplerate).T,axis=0)
      #print(chroma.shape)
      mel = np.mean(librosa.feature.melspectrogram(i, sr=samplerate).T,axis=0)
      #print(mel.shape)
      contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=samplerate).T,axis=0)
      #print(contrast.shape)
      tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(i), sr=samplerate).T,axis=0)
      #print(tonnetz.shape)
      ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
      mfcc_data.append(ext_features)#(1-d of 185 elements)
    c+=1

1 2133
2 2133
3 2133
4 2133
5 2133
6 2133
7 2133
8 2133
9 2133
10 2133
11 2133
12 2133
13 2133
14 2133
15 2133
16 2133
17 2133
18 2133
19 2133
20 2133
21 2133
22 2133
23 2133
24 2133
25 2133
26 2133
27 2133
28 2133
29 2133
30 2133
31 2133
32 2133
33 2133
34 2133
35 2133
36 2133
37 2133
38 2133
39 2133
40 2133
41 2133
42 2133
43 2133
44 2133
45 2133
46 2133
47 2133
48 2133
49 2133
50 2133
51 2133
52 2133
53 2133
54 2133
55 2133
56 2133
57 2133
58 2133
59 2133
60 2133
61 2133
62 2133
63 2133
64 2133
65 2133
66 2133
67 2133
68 2133
69 2133
70 2133
71 2133
72 2133
73 2133
74 2133
75 2133
76 2133
77 2133
78 2133
79 2133
80 2133
81 2133
82 2133
83 2133
84 2133
85 2133
86 2133
87 2133
88 2133
89 2133
90 2133
91 2133
92 2133
93 2133
94 2133
95 2133
96 2133
97 2133
98 2133
99 2133
100 2133
101 2133
102 2133
103 2133
104 2133
105 2133
106 2133
107 2133
108 2133
109 2133
110 2133
111 2133
112 2133
113 2133
114 2133
115 2133
116 2133
117 2133
118 2133
119 2133
120 2133
121 2133
122 2133
123 2133
1

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


233 2133
234 2133
235 2133
236 2133
237 2133
238 2133
239 2133
240 2133
241 2133
242 2133
243 2133
244 2133
245 2133
246 2133
247 2133
248 2133
249 2133
250 2133
251 2133
252 2133
253 2133
254 2133
255 2133
256 2133
257 2133
258 2133
259 2133
260 2133
261 2133
262 2133
263 2133
264 2133
265 2133
266 2133
267 2133
268 2133
269 2133
270 2133
271 2133
272 2133
273 2133
274 2133
275 2133
276 2133
277 2133
278 2133
279 2133
280 2133
281 2133
282 2133
283 2133
284 2133
285 2133
286 2133
287 2133
288 2133
289 2133
290 2133
291 2133
292 2133
293 2133
294 2133
295 2133
296 2133
297 2133
298 2133
299 2133
300 2133
301 2133
302 2133
303 2133
304 2133
305 2133
306 2133
307 2133
308 2133
309 2133
310 2133
311 2133
312 2133
313 2133
314 2133
315 2133
316 2133
317 2133
318 2133
319 2133
320 2133
321 2133
322 2133
323 2133
324 2133
325 2133
326 2133
327 2133
328 2133
329 2133
330 2133
331 2133
332 2133
333 2133
334 2133
335 2133
336 2133
337 2133
338 2133
339 2133
340 2133
341 2133
342 2133
343 2133
3

In [0]:
np.save("combined_feat.npy",mfcc_data)